In [31]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

regressors = [
    LinearRegression(),
    KNeighborsRegressor(),
    RandomForestRegressor(),
    DecisionTreeRegressor(),
]

df = pd.read_csv('/Users/rohith/Developer/rguhackcode/rguhack/project/GlobalE&C RGUHack Challenge/output_datasetAK47.csv')
# Selecting features and target for the regression model
features = ['mileage', 'mpg', 'tax', 'enginesize', 'maintenancecostyearly', 'fuel_cost_per_mile']
target = 'price'

# Prepare the data for training and testing
X = df[features]
y = df[target]


In [32]:
# One-hot encoding the 'fuelType', 'make', 'model', and 'transmission' columns
df_encoded = pd.get_dummies(df, columns=['make', 'model', 'transmission', 'fueltype'])

# Now df_encoded has the original columns, plus the one-hot encoded columns
# There is no need to concatenate df with dff or apply LabelEncoder on the one-hot encoded dataframe

# You can check the new dataframe to ensure the encoding is correct
print(df_encoded.head())



   year  price  mileage  tax   mpg  enginesize  maintenancecostyearly  \
0  2020  22500     2500  145  52.3         1.6                 477.34   
1  2017  11750    10326  145  68.9         1.6                 469.22   
2  2019  11699    20350  145  51.4         1.4                 389.31   
3  2019  31299     4831  145  61.4         2.0                 721.93   
4  2019  31450     4568  145  41.5         2.0                 662.21   

   fuel_cost_per_mile  purchase_cost    fuel_cost  ...  model_220  \
0            0.160017          22500  6400.699044  ...      False   
1            0.121465          11750  4858.585776  ...      False   
2            0.153767          11699  6150.665058  ...      False   
3            0.136302          31299  5452.061238  ...      False   
4            0.190448          31450  7617.932145  ...      False   

   transmission_automatic  transmission_manual  transmission_other  \
0                   False                False               False   
1     

In [17]:


# Create a label_encoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['make'] = label_encoder.fit_transform(df['make'])
df['model'] = label_encoder.fit_transform(df['model'])
df['transmission'] = label_encoder.fit_transform(df['transmission'])
df



,make,model,year,price,transmission,mileage,fueltype,tax,mpg,enginesize,maintenancecostyearly,fuel_cost_per_mile,purchase_cost,fuel_cost,maintenance_cost,total_cost_of_ownership
0,5,85,2020,22500,3,2500,diesel,145,52.3,1.6,477.34,0.160017,22500,6400.699044,2386.70,31287.399044
1,5,100,2017,11750,1,10326,diesel,145,68.9,1.6,469.22,0.121465,11750,4858.585776,2346.10,18954.685776
2,7,22,2019,11699,1,20350,petrol,145,51.4,1.4,389.31,0.153767,11699,6150.665058,1946.55,19796.215058
3,4,29,2019,31299,3,4831,diesel,145,61.4,2.0,721.93,0.136302,31299,5452.061238,3609.65,40360.711238
4,1,2,2019,31450,3,4568,petrol,145,41.5,2.0,662.21,0.190448,31450,7617.932145,3311.05,42378.982145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50401,2,54,2019,16495,1,7137,petrol,150,60.1,1.0,393.52,0.131508,16495,5260.302562,1967.60,23722.902562
50402,3,74,2017,13495,1,300,petrol,145,52.3,1.4,503.71,0.151121,13495,6044.821874,2518.55,22058.371874
50403,2,50,2017,21999,1,22000,diesel,150,47.9,2.0,415.60,0.174716,21999,6988.654697,2078.00,31065.654697
50404,3,88,2018,12495,1,23387,petrol,145,52.3,1.0,382.33,0.151121,12495,6044.821874,1911.65,20451.471874


In [18]:
from sklearn.model_selection import train_test_split


# Assuming 'target' is the name of your target variable
X = df_encoded.drop(columns=[target])
y = df_encoded[target]

# Splitting the dataset into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=0.6, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, train_size=0.5, random_state=0)

print(y_train.shape)
print(y_valid.shape)
print(X_test.shape)


(30243,)
(10081,)
(10082, 192)


In [19]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error, r2_score

head = 7  # Number of models to evaluate

for model in regressors[:head]:
    model.fit(X_train, y_train)  # Train the model on the training data
    y_pred_valid = model.predict(X_valid)  # Predict on the validation data
    y_pred_train = model.predict(X_train)  # Predict on the training data for comparison

    # Print the model and various evaluation metrics
    print(model)
    print("\tExplained variance train:", explained_variance_score(y_train, y_pred_train))
    print("\tExplained variance valid:", explained_variance_score(y_valid, y_pred_valid))
    print()
    print("\tMean absolute error:", mean_absolute_error(y_valid, y_pred_valid))
    print('\tMean Squared Error:', mean_squared_error(y_valid, y_pred_valid))
    print("\tR2 score:", r2_score(y_valid, y_pred_valid))
    print()

LinearRegression()
	Explained variance train: 1.0
	Explained variance valid: 1.0

	Mean absolute error: 2.7665293305680304e-10
	Mean Squared Error: 2.3426922413016824e-16
	R2 score: 1.0

KNeighborsRegressor()
	Explained variance train: 0.9990784547630261
	Explained variance valid: 0.9993953442390233

	Mean absolute error: 155.84483682174385
	Mean Squared Error: 51693.211558377145
	R2 score: 0.9993949488496185

RandomForestRegressor()
	Explained variance train: 0.9999128527331443
	Explained variance valid: 0.9999935484793531

	Mean absolute error: 1.9495377442714068
	Mean Squared Error: 551.1944883940117
	R2 score: 0.9999935484592806

DecisionTreeRegressor()
	Explained variance train: 1.0
	Explained variance valid: 0.9999954145315892

	Mean absolute error: 2.1307409979168734
	Mean Squared Error: 391.7684753496677
	R2 score: 0.9999954144855863



In [21]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Assuming 'regressors' is the list of models you want to cross-validate
for model in regressors:
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    rmse_scores = np.sqrt(-scores)
    print(f"Model: {model.__class__.__name__}")
    print(f"RMSE scores: {rmse_scores}")
    print(f"Average RMSE: {np.mean(rmse_scores)}")
    print()

Model: LinearRegression
RMSE scores: [7.25431434e-09 2.33535314e-09 4.37412997e-09 1.00785775e-08
 7.36325944e-09]
Average RMSE: 6.281126874470489e-09

Model: KNeighborsRegressor
RMSE scores: [213.68614753 209.13478203 247.75214188 205.61826391 492.20251714]
Average RMSE: 273.6787704971658

Model: RandomForestRegressor
RMSE scores: [ 43.91014647  12.54025774 111.87294365  10.83037806 420.85696193]
Average RMSE: 120.00213756830524

Model: DecisionTreeRegressor
RMSE scores: [513.97930459  20.47362519 134.22122658  18.04494054 449.08251927]
Average RMSE: 227.16032323489608



In [26]:
## car logic
model = LinearRegression()
model.fit(X, y)



LinearRegression()

In [27]:
#car_type = input car brand name 
#runs ml model to get recomendations    
def predict_for_car_type(car_type):
    selected_cars = df[df['make'] == car_type].copy()
    if selected_cars.empty:
        return pd.DataFrame()
    
    usable_cars = selected_cars[selected_cars['mileage'] <= 8000 - (5 * selected_cars['mpg'])].copy()
    if usable_cars.empty:
        return pd.DataFrame()
    
    predicted_prices = xg_reg.predict(usable_cars[features])
    usable_cars.loc[:, 'predicted_price'] = predicted_prices
    return usable_cars.nsmallest(3, 'predicted_price')


def predict_and_compare(car_type):
    specified_cars = predict_for_car_type(car_type)
    if specified_cars.empty:
        print("No usable cars found for the specified type that are estimated to last for the next 5 years.")
        return
    
    print("Cheapest usable cars for", car_type + ":")
    print(specified_cars[['make', 'model', 'predicted_price']])
    
    best_alternatives = pd.DataFrame(columns=specified_cars.columns)
    for other_car_type in df['make'].unique():
        if other_car_type != car_type:
            alternatives = predict_for_car_type(other_car_type)
            if not alternatives.empty:
                cheapest_alternative = alternatives.nsmallest(1, 'predicted_price')
                best_alternatives = pd.concat([best_alternatives, cheapest_alternative])
    
    if not best_alternatives.empty:
        print("\nBest alternatives from other car types:")
        print(best_alternatives[['make', 'model', 'predicted_price']])
    else:
        print("\nNo better alternatives found from other types.")

car_type = input("Enter the type of car (e.g., audi, bmw, mercedes): ")

predict_and_compare(car_type)

No usable cars found for the specified type that are estimated to last for the next 5 years.


In [30]:
import pandas as pd

# Assuming df is your dataset DataFrame and you've already loaded it.
# user inputs
annual_mileage = float(input("Enter the annual mileage: "))
years_of_ownership = int(input("Enter the number of years you plan to own the car: "))

# Calculate annual fuel cost
df['annual_fuel_cost'] = df['fuel_cost_per_mile'] * annual_mileage

# Calculate total fuel cost for the period of ownership
df['total_fuel_cost'] = df['annual_fuel_cost'] * years_of_ownership

# Calculate total maintenance cost for the period of ownership
df['total_maintenance_cost'] = df['maintenancecostyearly'] * years_of_ownership

# Calculate total cost of ownership
df['total_cost_of_ownership'] = df['purchase_cost'] + df['total_fuel_cost'] + df['total_maintenance_cost']

# Optionally, sort the DataFrame by total cost of ownership to find the cheapest cars
df_sorted = df.sort_values('total_cost_of_ownership').reset_index(drop=True)

# Let's see the top 5 cars with the least total cost of ownership
top_5_cheapest_cars = df_sorted.head(5)
predicted_prices = model.predict(X[features])
model.loc[:, 'predicted_price'] = predicted_prices
top_3_predicted_cars = model.nsmallest(3, 'predicted_price')
top_3_predicted_cars





ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- fuel_cost
- fueltype_diesel
- fueltype_electric
- fueltype_hybrid
- fueltype_other
- ...
